In [1]:
import os
import io
import bz2
import gzip
import urllib
from tqdm import tqdm, trange
from itertools import combinations
import pandas as pd
from rdflib import Graph, URIRef, BNode, Literal
from rdflib.store import NO_STORE, VALID_STORE
from SPARQLWrapper import SPARQLWrapper, JSON

# 1. Extract properties to materialize

In [2]:
subproperties_file = "../Data/interim/DBpedia/properties/DBpedia_subproperties.csv"
inverse_properties_file = "../Data/interim/DBpedia/properties/DBpedia_Wikidata_mapped_inverse_prop.csv"
symmetric_properties_file = "../Data/interim/DBpedia/properties/DBpedia_Wikidata_mapped_symmetric_prop.csv"
transitive_properties_file = "../Data/interim/DBpedia/properties/DBpedia_Wikidata_mapped_transitive_prop.csv"
ontology_file = "../Data/raw/DBpedia/dbpedia_2016-10.nt"

In [3]:
def run_query(query):
    sparql = SPARQLWrapper("http://dbpedia.org/sparql")
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    results_df = pd.io.json.json_normalize(results["results"]["bindings"])
    return results_df

## 1.1. Extract subproperties

In [4]:
subproperties = pd.read_csv(subproperties_file)
subproperties.head()

,Subproperty,SubpropertyLabel,Property,PropertyLabel
0,http://dbpedia.org/ontology/championInDoubleFe...,champion in double female,http://dbpedia.org/ontology/championInDouble,champion in double
1,http://dbpedia.org/ontology/championInDouble,champion in double,http://dbpedia.org/ontology/champion,champion
2,http://dbpedia.org/ontology/maximumDepth,maximum depth (μ),http://dbpedia.org/ontology/depth,depth (μ)
3,http://dbpedia.org/ontology/eastPlace,east place,http://dbpedia.org/ontology/closeTo,is close to
4,http://dbpedia.org/ontology/meshNumber,MeSH number,http://dbpedia.org/ontology/code,code


In [5]:
subproperties_count = len(subproperties)
print("Queried {} subproperties from DBPedia.".format(subproperties_count))

Queried 75 subproperties from DBPedia.


## 1.2. Extract inverse properties

In [6]:
inverse_properties = pd.read_csv(inverse_properties_file)
inverse_properties.head()

,DBpediaProp,DBPediaPropLabel,WikidataProp,WikidataPropLabel,DBpediaInvProp,DBpediaInvPropLabel,WikidataInvProp,WikidataInvPropLabel
0,http://dbpedia.org/ontology/child,child,http://www.wikidata.org/entity/P40,child,http://dbpedia.org/ontology/father,father,http://www.wikidata.org/entity/P22,father
1,http://dbpedia.org/ontology/child,child,http://www.wikidata.org/entity/P40,child,http://dbpedia.org/ontology/mother,mother,http://www.wikidata.org/entity/P25,mother
2,http://dbpedia.org/ontology/father,father,http://www.wikidata.org/entity/P22,father,http://dbpedia.org/ontology/child,child,http://www.wikidata.org/entity/P40,child
3,http://dbpedia.org/ontology/mother,mother,http://www.wikidata.org/entity/P25,mother,http://dbpedia.org/ontology/child,child,http://www.wikidata.org/entity/P40,child
4,http://dbpedia.org/ontology/followedBy,followed by,http://www.wikidata.org/entity/P156,followed by,http://dbpedia.org/ontology/follows,follows,http://www.wikidata.org/entity/P155,follows


In [7]:
inverse_properties_count = len(inverse_properties)
print("Queried {} inverse properties from DBPedia.".format(inverse_properties_count))

Queried 8 inverse properties from DBPedia.


## 1.3. Extract symmetric properties

In [8]:
symmetric_properties = pd.read_csv(symmetric_properties_file)
symmetric_properties.head()

,DBpediaProp,DBpediaPropLabel,WikidataProp,WikidataPropLabel


In [9]:
symmetric_properties_count = len(symmetric_properties)
print("Queried {} symmetric properties from DBPedia.".format(symmetric_properties_count))

Queried 0 symmetric properties from DBPedia.


In [10]:
sym_prop = ["http://dbpedia.org/ontology/neighboringMunicipality", 
            "http://dbpedia.org/ontology/sisterCollege", 
            "http://dbpedia.org/ontology/currentPartner"]
symmetric_properties = pd.DataFrame(sym_prop, columns = ["DBpediaProp"])
symmetric_properties

,DBpediaProp
0,http://dbpedia.org/ontology/neighboringMunicip...
1,http://dbpedia.org/ontology/sisterCollege
2,http://dbpedia.org/ontology/currentPartner


In [11]:
symmetric_properties_count = len(symmetric_properties)
print("Queried {} symmetric properties from DBPedia.".format(symmetric_properties_count))

Queried 3 symmetric properties from DBPedia.


## 1.4. Extract transitive properties

In [12]:
transitive_properties = pd.read_csv(transitive_properties_file)
transitive_properties.head()

,DBpediaProp,DBpediaPropLabel,WikidataProp,WikidataPropLabel
0,http://dbpedia.org/ontology/city,city,http://www.wikidata.org/entity/P131,located in the administrative territorial entity
1,http://dbpedia.org/ontology/county,county,http://www.wikidata.org/entity/P131,located in the administrative territorial entity
2,http://dbpedia.org/ontology/district,district,http://www.wikidata.org/entity/P131,located in the administrative territorial entity
3,http://dbpedia.org/ontology/isPartOf,is part of,http://www.wikidata.org/entity/P361,part of
4,http://dbpedia.org/ontology/locatedInArea,located in area,http://www.wikidata.org/entity/P131,located in the administrative territorial entity


In [13]:
transitive_properties_count = len(transitive_properties)
print("Queried {} transitive properties from DBPedia.".format(transitive_properties_count))

Queried 7 transitive properties from DBPedia.


# 2. Enrich graph with extracted properties

## 2.1. Construct graph

In [14]:
dbpedia_file_instances = "../Data/raw/DBpedia/instance_types_transitive_en.ttl.bz2"
dbpedia_file_objects = "../Data/raw/DBpedia/mappingbased_objects_en.ttl.bz2"
dbpedia_graph = "../Data/interim/DBpedia/graph/dbpedia_graph.nt.gz"
dbpedia_enriched_graph = "../Data/interim/DBpedia/graph/dbpedia_enriched_graph.nt.gz"
path_store = "../Data/interim/DBpedia/graph/DBpediaLibStore"

In [15]:
def quote(string):
    return urllib.parse.quote(string, encoding = "utf-8", safe = ":/%#")

def create_node(string):
    if string.startswith("<"):
        if string[-2] != "/":
            return URIRef(quote(string[1:-1]))
        else:
            return URIRef(quote(string[1:-2]))
    elif string.startswith('"'):
        return Literal(string[1:string.rindex('"')])
    elif string.startswith("_:"):
        return BNode(string[2:])
    else:
        return "Invalid node type"

In [17]:
# Load file into graph
print("Reading file and creating graph.")

# Open previously created store, or create it if it doesn't exist yet
graph = Graph("Sleepycat")

rt = graph.open(path_store, create=False)

if rt == NO_STORE:
    # There is no underlying Sleepycat infrastructure, create it
    graph.open(path_store, create = True)
else:
    assert rt == VALID_STORE, "The underlying store is corrupt."
    
print('Triples in graph before add: ', len(graph))

with tqdm(desc="Reading triples and adding to graph: ", total = 4529682753, unit = "bytes") as pbar:
    with bz2.open(filename=dbpedia_file_instances, encoding="utf-8", mode="rt") as f:
        for line in f:
            pbar.update(len(line))
            if line not in ['\n', '\r\n'] and "# started " not in line and "# completed" not in line:
                triples = line.split(" ", maxsplit=2)
                subj = create_node(triples[0])
                pred = create_node(triples[1])
                obj = create_node(triples[2].rsplit(" .")[0])
                graph.add((subj,pred,obj))
                        
        
print('Triples in graph after adding instances: ', len(graph))

with tqdm(desc="Reading triples and adding to graph: ", total = 2576292481, unit = "bytes") as pbar:
    with bz2.open(filename=dbpedia_file_objects, encoding="utf-8", mode="rt") as f:
        for line in f:
            pbar.update(len(line))
            if line not in ['\n', '\r\n'] and "# started " not in line and "# completed" not in line:
                triples = line.split(" ", maxsplit=2)
                subj = create_node(triples[0])
                pred = create_node(triples[1])
                obj = create_node(triples[2].rsplit(" .")[0])
                graph.add((subj,pred,obj))
                
print('Triples in graph after adding objects: ', len(graph))

Reading triples and adding to graph:   0%|          | 0/4529682753 [00:00<?, ?bytes/s]

Reading file and creating graph.
Triples in graph before add:  0


Reading triples and adding to graph:   0%|          | 0/2576292481 [00:00<?, ?bytes/s]

Triples in graph after adding instances:  31254270


Reading triples and adding to graph: 100%|██████████| 2576292481/2576292481 [2:12:39<00:00, 323670.65bytes/s]  


Triples in graph after adding objects:  50000412


## 2.2. Saving graph

In [18]:
print("Serializing and saving graph.")
with gzip.open(dbpedia_graph, "wb") as f:
    graph.serialize(destination=f, format="nt")
print("Graph saved.")

Serializing and saving graph.
Graph saved.


## 2.3. Materialize subproperties

In [35]:
count_subproperties = 0
for idx in trange(len(subproperties), desc = "Iterating over subproperties: ", unit = "subproperties"):
    subProp = URIRef(subproperties["Subproperty"].loc[idx])
    prop = URIRef(subproperties["Property"].loc[idx])
    instances = list(graph.subject_objects(predicate=subProp))
    for row in range(len(instances)):
        lnode = instances[row][0]
        rnode = instances[row][1]
        if not ((rnode, prop, lnode)) in graph:
            graph.add((rnode, prop, lnode))
            count_subproperties += 1

Iterating over subproperties: 100%|██████████| 75/75 [00:17<00:00,  4.50subproperties/s]


In [36]:
print("Added {} new triples by materializing subproperties.".format(count_subproperties))
print("The graph contains {} triples after materializing subproperties.".format(len(graph)))

Added 0 new triples by materializing subproperties.
The graph contains 50506250 triples after materializing subproperties.


## 2.4. Materialize inverse properties

In [37]:
count_inverse = 0
for idx in trange(len(inverse_properties), desc="Iterating over inverse properties: ", unit = "inverse properties"):
    prop = URIRef(inverse_properties["DBpediaProp"].loc[idx])
    inverse_prop = URIRef(inverse_properties["DBpediaInvProp"].loc[idx])
    if prop != "http://dbpedia.org/ontology/child":
        instances = list(graph.subject_objects(predicate=prop))
        for row in range(len(instances)):
            lnode = instances[row][0]
            rnode = instances[row][1]
            if not ((rnode, inverse_prop, lnode)) in graph:
                graph.add((rnode, inverse_prop, lnode))
                count_inverse += 1
    else:
        instances = list(graph.subject_objects(predicate=inverse_prop))
        for row in range(len(instances)):
            lnode = instances[row][0]
            rnode = instances[row][1]
            if not ((rnode, prop, lnode)) in g:
                graph.add((rnode, prop, lnode))
                count_inverse += 1

Iterating over inverse properties: 100%|██████████| 8/8 [00:15<00:00,  1.69s/inverse properties]


In [38]:
print("Added {} new triples by materializing inverse properties.".format(count_inverse))
print("The graph contains {} triples after materializing inverse properties.".format(len(graph)))

Added 0 new triples by materializing inverse properties.
The graph contains 50506250 triples after materializing inverse properties.


## 2.5. Materialize symmetric properties

In [39]:
count_symmetric = 0
for symProp in symmetric_properties["DBpediaProp"]:
    prop = URIRef(symProp)
    print(prop)
    instances = list(graph.subject_objects(predicate = prop))
    for row in range(len(instances)):
        lnode = URIRef(instances[row][1])
        rnode = URIRef(instances[row][0])
        if not (lnode, prop, rnode) in graph:
            graph.add((lnode, prop, rnode))
            count_symmetric += 1

http://dbpedia.org/ontology/neighboringMunicipality
http://dbpedia.org/ontology/sisterCollege
http://dbpedia.org/ontology/currentPartner


In [40]:
print("Added {} new triples by materializing symmetric properties.".format(count_symmetric))
print("The graph contains {} triples after materializing symmetric properties.".format(len(graph)))

Added 0 new triples by materializing symmetric properties.
The graph contains 50506250 triples after materializing symmetric properties.


## 2.6. Materialize transitive properties

In [41]:
count_transitive = 0
for row in trange(len(transitive_properties), desc="Iterating over transitive properties: ", unit="transitive properties"):
    prop = URIRef(transitive_properties["DBpediaProp"].loc[row])
    print(prop)
    instances = set(graph.subjects(predicate=prop))
    with tqdm(desc="Iterating over instances: ", total = len(instances), unit = "instances") as pbar:
        for root in instances:
            pbar.update(1)
            transitive_objects = list(graph.transitive_objects(property=prop, subject=root))
            for new_node in transitive_objects:
                if root != new_node:
                    if not (root, prop, new_node) in graph:
                        graph.add((root, prop, new_node))
                        count_transitive += 1                    

Iterating over transitive properties:   0%|          | 0/7 [00:00<?, ?transitive properties/s]

http://dbpedia.org/ontology/city



Iterating over instances:  52%|█████▏    | 45671/87306 [00:18<00:15, 2621.56instances/s]


Iterating over transitive properties:  14%|█▍        | 1/7 [00:39<03:58, 39.80s/transitive properties]

http://dbpedia.org/ontology/county


...

http://dbpedia.org/ontology/locatedInArea



Iterating over transitive properties:  71%|███████▏  | 5/7 [09:21<03:58, 119.41s/transitive properties]

http://dbpedia.org/ontology/province



Iterating over transitive properties: 100%|██████████| 7/7 [09:28<00:00, 81.22s/transitive properties]

http://dbpedia.org/ontology/settlement


In [42]:
print("Added {} new triples by materializing transitive properties.".format(count_transitive))
print("The graph contains {} triples after materializing transitive properties.".format(len(graph)))

Added 0 new triples by materializing transitive properties.
The graph contains 50506250 triples after materializing transitive properties.


## 2.7. Save enriched graph

In [43]:
print("Enriched graph saved. The graph contains {} triples.".format(len(graph)))

Enriched graph saved. The graph contains 50506250 triples.


In [44]:
print("Serializing and saving graph.")
with gzip.open(dbpedia_enriched_graph, "wb") as f:
    graph.serialize(destination=f, format="nt")
print("Graph saved.")

Serializing and saving graph.
Graph saved.


In [45]:
graph.close()

In [46]:
print("Deleting temporary files.")
for f in os.listdir(path_store):
    os.unlink(path_store + '/' + f)
os.rmdir(path_store)

Deleting temporary files.
